# Check metadata

* Purpose of script: This notebook will transform netCDF to geotiff.  
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170731

as explained in the previous step, we need to put the NetCDF data into a coordinate reference system. For that we use a standard geometry. 

Download the standard geotiff to our instance:

# Preparation

Run the following command just in case an old folder exists. _If no older folder exist you will get the error: rm: cannot remove '/volumes/data/PCRGlobWB20V01/additional/*': No such file or directory_

In [1]:
!rm -r /volumes/data/PCRGlobWB20V01/additional/*

rm: cannot remove '/volumes/data/PCRGlobWB20V01/additional/*': No such file or directory


In [2]:
!aws s3 sync s3://wri-projects/Aqueduct30/rawData/WRI/samplegeotiff/ /volumes/data/PCRGlobWB20V01/additional/

download: s3://wri-projects/Aqueduct30/rawData/WRI/samplegeotiff/readme.txt to ../../../../data/PCRGlobWB20V01/additional/readme.txt
download: s3://wri-projects/Aqueduct30/rawData/WRI/samplegeotiff/sampleGeotiff.tiff to ../../../../data/PCRGlobWB20V01/additional/sampleGeotiff.tiff


Check if the file is actually copied

In [3]:
!rm -r /volumes/data/temp*

In [4]:
!mkdir /volumes/data/temp

In [5]:
!ls /volumes/data/PCRGlobWB20V01/additional/

readme.txt  sampleGeotiff.tiff


In [6]:
try:
    from osgeo import ogr, osr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')
    
from netCDF4 import Dataset
import os
import datetime
import subprocess

# Settings

In [7]:
NETCDFINPUTPATH = "/volumes/data/PCRGlobWB20V01/"
PRINT_METADATA = False
OUTPUTPATH = "/volumes/data/Y2017M07D31_RH_Convert_NetCDF_Geotiff_V01/"
inputLocationSampleGeotiff = "/volumes/data/PCRGlobWB20V01/additional/sampleGeotiff.tiff"

In [8]:
!mkdir /volumes/data/Y2017M07D31_RH_Convert_NetCDF_Geotiff_V01

mkdir: cannot create directory '/volumes/data/Y2017M07D31_RH_Convert_NetCDF_Geotiff_V01': File exists


# Functions

In [9]:
def netCDF4toGeotiff(fileName,fileLocation):
    netCDFInputBaseName = fileName.split('.')[0]
    nc_fid = Dataset(fileLocation, 'r')
    nc_attrs, nc_dims, nc_vars = ncdump(nc_fid, PRINT_METADATA)
    parameter = nc_vars[3]
    lats = nc_fid.variables['latitude'][:]  # extract/copy the data
    lons = nc_fid.variables['longitude'][:]
    times = nc_fid.variables['time'][:]
    timeUnit = nc_fid.variables["time"].getncattr("units")
    timeNormal =[]
    for time in times:
        if timeUnit == ("days since 1900-01-01 00:00:00") or (timeUnit =="Days since 1900-01-01"):
            timeNormal.append(datetime.datetime(1900,1,1) + datetime.timedelta(days=time))
        elif timeUnit == "days since 1901-01-01 00:00:00":
            timeNormal.append(datetime.datetime(1901,1,1) + datetime.timedelta(days=time))
        else:
            print "Error"
            timeNormal.append(-9999)
            
    for i in range(0,len(timeNormal)):
        print timeNormal[i].year
        Z = nc_fid.variables[parameter][i, :, :]
        Z[Z<-9990]= -9999
        Z[Z>1e19] = -9999
        outputFilename = netCDFInputBaseName + "I%0.3dY%0.2dM%0.2d.tif" %(i,timeNormal[i].year,timeNormal[i].month)
        writefilename = os.path.join(OUTPUTPATH,outputFilename)
        writeFile(writefilename,geotransform,geoproj,Z)
    
    return time, timeUnit, timeNormal

def readFile(filename):
    filehandle = gdal.Open(filename)
    band1 = filehandle.GetRasterBand(1)
    geotransform = filehandle.GetGeoTransform()
    geoproj = filehandle.GetProjection()
    Z = band1.ReadAsArray()
    xsize = filehandle.RasterXSize
    ysize = filehandle.RasterYSize
    filehandle = None
    return xsize,ysize,geotransform,geoproj,Z

def writeFile(filename,geotransform,geoprojection,data):
    (x,y) = data.shape
    format = "GTiff"
    driver = gdal.GetDriverByName(format)
    # you can change the dataformat but be sure to be able to store negative values including -9999
    dst_datatype = gdal.GDT_Float32
    dst_ds = driver.Create(filename,y,x,1,dst_datatype, [ 'COMPRESS=LZW' ])
    dst_ds.GetRasterBand(1).SetNoDataValue(-9999)
    dst_ds.GetRasterBand(1).WriteArray(data)
    dst_ds.SetGeoTransform(geotransform)
    dst_ds.SetProjection(geoprojection)
    dst_ds = None
    return 1

def ncdump(nc_fid, verb=True):
    '''
    ncdump outputs dimensions, variables and their attribute information.
    The information is similar to that of NCAR's ncdump utility.
    ncdump requires a valid instance of Dataset.

    Parameters
    ----------
    nc_fid : netCDF4.Dataset
        A netCDF4 dateset object
    verb : Boolean
        whether or not nc_attrs, nc_dims, and nc_vars are printed

    Returns
    -------
    nc_attrs : list
        A Python list of the NetCDF file global attributes
    nc_dims : list
        A Python list of the NetCDF file dimensions
    nc_vars : list
        A Python list of the NetCDF file variables
    '''
    def print_ncattr(key):
        """
        Prints the NetCDF file attributes for a given key

        Parameters
        ----------
        key : unicode
            a valid netCDF4.Dataset.variables key
        """
        try:
            print "\t\ttype:", repr(nc_fid.variables[key].dtype)
            for ncattr in nc_fid.variables[key].ncattrs():
                print '\t\t%s:' % ncattr,\
                      repr(nc_fid.variables[key].getncattr(ncattr))
        except KeyError:
            print "\t\tWARNING: %s does not contain variable attributes" % key

    # NetCDF global attributes
    nc_attrs = nc_fid.ncattrs()
    if verb:
        print "NetCDF Global Attributes:"
        for nc_attr in nc_attrs:
            print '\t%s:' % nc_attr, repr(nc_fid.getncattr(nc_attr))
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions
    # Dimension shape information.
    if verb:
        print "NetCDF dimension information:"
        for dim in nc_dims:
            print "\tName:", dim
            print "\t\tsize:", len(nc_fid.dimensions[dim])
            print_ncattr(dim)
    # Variable information.
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables
    if verb:
        print "NetCDF variable information:"
        for var in nc_vars:
            if var not in nc_dims:
                print '\tName:', var
                print "\t\tdimensions:", nc_fid.variables[var].dimensions
                print "\t\tsize:", nc_fid.variables[var].size
                print_ncattr(var)
    return nc_attrs, nc_dims, nc_vars

# Script

In [10]:
[xsize,ysize,geotransform,geoproj,ZSample] = readFile(inputLocationSampleGeotiff)

These are the parameters of the standard geometry. 

In [11]:
print xsize, ysize, geotransform

4320 2160 (-179.99999491255934, 0.0833333309780367, 0.0, 90.00000254430942, 0.0, -0.0833333309780367)


In [12]:
for root, dirs, files in os.walk(NETCDFINPUTPATH):
    for oneFile in files:
        if oneFile.endswith(".nc4"):
            fileLocation = os.path.join(root, oneFile)
            fileName = oneFile
            netCDF4toGeotiff(fileName,fileLocation)
                

22249.0
22614.0
22979.0
23344.0
23710.0
24075.0
24440.0
24805.0
25171.0
25536.0
25901.0
26266.0
26632.0
26997.0
27362.0
27727.0
28093.0
28458.0
28823.0
29188.0
29554.0
29919.0
30284.0
30649.0
31015.0
31380.0
31745.0
32110.0
32476.0
32841.0
33206.0
33571.0
33937.0
34302.0
34667.0
35032.0
35398.0
35763.0
36128.0
36493.0
36859.0
37224.0
37589.0
37954.0
38320.0
38685.0
39050.0
39415.0
39781.0
40146.0
40511.0
40876.0
41242.0
41607.0
41972.0
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
21914.0
21945.0
21974.0
22005.0
22035.0
22066.0
22096.0
22127.0
22158.0
22188.0
22219.0
22249.0
22280.0
22311.0
22339.0
22370.0
22400.0
22431.0
22461.0
22492.0
22523.0
22553.0
22584.0
22614.0
22645.0
22676.0
22704.0
22735.0
22765.0
22796.0
22826.0
22857.0
22888.0
22918.0
22949.0
22979

1996
1996
1996
1996
1997
1997
1997
1997
1997
1997
1997
1997
1997
1997
1997
1997
1998
1998
1998
1998
1998
1998
1998
1998
1998
1998
1998
1998
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2001
2001
2001
2001
2001
2001
2001
2001
2001
2001
2001
2001
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009
2010
2010
2010
2010
2010
2010
2010
2010
2010
2010
2010
2010
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011
2011
2012
2012
2012
2012
2012
2012
2012
2012
2012
2012
2012
2012
2013
2013
2013
2013


1990
1990
1990
1990
1990
1990
1990
1990
1990
1990
1990
1990
1991
1991
1991
1991
1991
1991
1991
1991
1991
1991
1991
1991
1992
1992
1992
1992
1992
1992
1992
1992
1992
1992
1992
1992
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1996
1996
1996
1996
1996
1996
1996
1996
1996
1996
1996
1996
1997
1997
1997
1997
1997
1997
1997
1997
1997
1997
1997
1997
1998
1998
1998
1998
1998
1998
1998
1998
1998
1998
1998
1998
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2001
2001
2001
2001
2001
2001
2001
2001
2001
2001
2001
2001
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2006
2006
2006
2006
2006
2006
2006
2006


1983
1983
1983
1983
1983
1983
1983
1984
1984
1984
1984
1984
1984
1984
1984
1984
1984
1984
1984
1985
1985
1985
1985
1985
1985
1985
1985
1985
1985
1985
1985
1986
1986
1986
1986
1986
1986
1986
1986
1986
1986
1986
1986
1987
1987
1987
1987
1987
1987
1987
1987
1987
1987
1987
1987
1988
1988
1988
1988
1988
1988
1988
1988
1988
1988
1988
1988
1989
1989
1989
1989
1989
1989
1989
1989
1989
1989
1989
1989
1990
1990
1990
1990
1990
1990
1990
1990
1990
1990
1990
1990
1991
1991
1991
1991
1991
1991
1991
1991
1991
1991
1991
1991
1992
1992
1992
1992
1992
1992
1992
1992
1992
1992
1992
1992
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1993
1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1994
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1995
1996
1996
1996
1996
1996
1996
1996
1996
1996
1996
1996
1996
1997
1997
1997
1997
1997
1997
1997
1997
1997
1997
1997
1997
1998
1998
1998
1998
1998
1998
1998
1998
1998
1998
1998
1998
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
2000


1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1976
1976
1976
1976
1976
1976
1976
1976
1976


...

1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1976
1976
1976
1976
1976
1976
1976
1976
1976


1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1976
1976
1976
1976
1976
1976
1976
1976
1976


1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1976
1976
1976
1976
1976
1976
1976
1976
1976


1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1976
1976
1976
1976
1976
1976
1976
1976
1976


1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1960
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1961
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1962
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1963
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1964
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1965
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1966
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1967
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1968
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1969
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1970
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1971
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1972
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1973
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1974
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1975
1976
1976
1976
1976
1976
1976
1976
1976
1976


In [13]:
files = os.listdir(OUTPUTPATH)
print("Number of files: " + str(len(files)))

Number of files: 8552


Some files from Utrecht contain double years, removing the erroneous ones (used Panoply/Qgis to inspect those files):

global_historical_PDomWN_year_millionm3_5min_1960_2014I055Y1960M01.tif
global_historical_PDomWN_month_millionm3_5min_1960_2014I660Y1960M01.tif
global_historical_PDomWN_month_millionm3_5min_1960_2014I661Y1960M01.tif




In [15]:
!mkdir /volumes/data/trash

mkdir: cannot create directory '/volumes/data/trash': File exists


In [16]:
!mv /volumes/data/Y2017M07D31_RH_Convert_NetCDF_Geotiff_V01/global_historical_PDomWN_year_millionm3_5min_1960_2014I055Y1960M01.tif /volumes/data/trash/global_historical_PDomWN_year_millionm3_5min_1960_2014I055Y1960M01.tif
!mv /volumes/data/Y2017M07D31_RH_Convert_NetCDF_Geotiff_V01/global_historical_PDomWN_month_millionm3_5min_1960_2014I660Y1960M01.tif /volumes/data/trash/global_historical_PDomWN_month_millionm3_5min_1960_2014I660Y1960M01.tif
!mv /volumes/data/Y2017M07D31_RH_Convert_NetCDF_Geotiff_V01/global_historical_PDomWN_month_millionm3_5min_1960_2014I661Y1960M01.tif /volumes/data/trash/global_historical_PDomWN_month_millionm3_5min_1960_2014I661Y1960M01.tif



In [17]:
files = os.listdir(OUTPUTPATH)
print("Number of files: " + str(len(files)))

Number of files: 8549
